In [1]:
!nvidia-smi

Fri Nov 24 07:10:32 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8    10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
%cd '/content/drive/MyDrive/YoloV8'

/content/drive/MyDrive/YoloV8


In [3]:
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 645.7/645.7 kB 12.1 MB/s eta 0:00:00


## Creating train-val split

In [4]:
import os
import shutil
import random

!pip install tqdm --upgrade
from tqdm.notebook import tqdm

## Creating path for train and validation image and labels and test images


In [5]:
train_path_img = "yolo_data/images/train/"
train_path_label = "yolo_data/labels/train/"
val_path_img = "yolo_data/images/val/"
val_path_label = "yolo_data/labels/val/"
test_path = "yolo_data/test"

In [6]:
"""
Split the dataset into train and test creates train.txt and test.txt with the
respective path of the images in each folder
"""
def train_test_split(path, neg_path=None, split=0.2):
    print("---------- PROCESS STARTED ----------")
    files = list(set([name[:-4] for name in os.listdir(path)]))
    print(f"-- This folder has total number of {len(files)} images --")
    random.seed(42)
    random.shuffle(files)

    test_size = int(len(files)*split)
    train_size = len(files) - test_size

    # creating required directories
    os.makedirs(train_path_img, exist_ok = True)
    os.makedirs(train_path_label, exist_ok = True)
    os.makedirs(val_path_img, exist_ok = True)
    os.makedirs(val_path_label, exist_ok = True)

    ### ---- copying images to train folder
    for filex in tqdm(files[:train_size]):
        if filex == "classes":
            continue
        shutil.copy2(path + filex +'.jpg',f"{train_path_img}/"+ filex +'.jpg')
        shutil.copy2(path + filex + '.txt', f"{train_path_label}/"+ filex + '.txt')

    print(f'------ Training data created with 80% split {len(files[:train_size])} images -------')
    if neg_path:
        neg_images = list(set([name[:-4] for name in os.listdir(neg_path)]))
        for filex in tqdm(neg_images):
            shutil.copy2(neg_path+filex+".jpg", f"{train_path_img}/" + filex + '.jpg')

        print(f'----- Total {len(neg_images)} negative images added to the training data -----')
        print(f'----- TOTAL Training data created with {len(files[:train_size]) + len(neg_images)} images------')

    for filex in tqdm(files[train_size:]):
        if filex == "classes":
            continue
        print('running....')
        shutil.copy2(path + filex + '.jpg', f"{val_path_img}/" + filex + '.jpg')
        shutil.copy2(path + filex + '.txt', f"{val_path_label}/" + filex + ".txt")
    print(f"-------- Testing data created with a total of {len(files[train_size:])} images ----------")
    print("-------- TASK COMPLETED ---------")


train_test_split("/content/drive/MyDrive/YoloV8/data/")

---------- PROCESS STARTED ----------
-- This folder has total number of 66 images --


  0%|          | 0/53 [00:00<?, ?it/s]

------ Training data created with 80% split 53 images -------


  0%|          | 0/13 [00:00<?, ?it/s]

running....
running....
running....
running....
running....
running....
running....
running....
running....
running....
running....
running....
running....
-------- Testing data created with a total of 13 images ----------
-------- TASK COMPLETED ---------


In [7]:
open("/content/drive/MyDrive/YoloV8/dataset.yaml")

<_io.TextIOWrapper name='/content/drive/MyDrive/YoloV8/dataset.yaml' mode='r' encoding='UTF-8'>

In [8]:
import ultralytics
ultralytics.checks()

Ultralytics YOLOv8.0.216 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.1/78.2 GB disk)


## Training

In [10]:
!yolo task = detect mode = train model = yolov8s.pt data =/content/drive/MyDrive/YoloV8/dataset.yaml epochs= 10 imgsz = 640 batch=8 project=/content/drive/MyDrive/YoloV8/training_results name=football

100% 21.5M/21.5M [00:00<00:00, 175MB/s]
Ultralytics YOLOv8.0.216 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/drive/MyDrive/YoloV8/dataset.yaml, epochs=10, patience=50, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=/content/drive/MyDrive/YoloV8/training_results, name=football, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, stream_buffer=False, line_width=None, visualize=False, augment=False, agn

## Inferencing

In [12]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/YoloV8/training_results/football/weights/best.pt conf=0.35 source=/content/drive/MyDrive/YoloV8/test_images

Ultralytics YOLOv8.0.216 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

image 1/3 /content/drive/MyDrive/YoloV8/test_images/pexels-anastasia-shuraeva-9442066.jpg: 448x640 18 players, 106.3ms
image 2/3 /content/drive/MyDrive/YoloV8/test_images/pexels-rdne-stock-project-7188092.jpg: 640x448 9 players, 63.8ms
image 3/3 /content/drive/MyDrive/YoloV8/test_images/pexels-yahye-abdi-10512973.jpg: 448x640 5 players, 3 footballs, 13.6ms
Speed: 2.9ms preprocess, 61.3ms inference, 29.7ms postprocess per image at shape (1, 3, 448, 640)
Results saved to runs/detect/predict2
💡 Learn more at https://docs.ultralytics.com/modes/predict


## Copying results to output folder

In [13]:
!cp -r /content/drive/MyDrive/YoloV8/runs/detect/predict2 /content/drive/MyDrive/YoloV8/output

## Inferencing on video

In [14]:
!yolo task=detect mode=predict model=/content/drive/MyDrive/YoloV8/training_results/football/weights/best.pt conf=0.77 source=/content/drive/MyDrive/YoloV8/videos


Ultralytics YOLOv8.0.216 🚀 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11126358 parameters, 0 gradients, 28.4 GFLOPs

video 1/4 (1/376) /content/drive/MyDrive/YoloV8/videos/pexels-anastasia-shuraeva-9429927 (720p).mp4: 640x352 (no detections), 95.4ms
video 1/4 (2/376) /content/drive/MyDrive/YoloV8/videos/pexels-anastasia-shuraeva-9429927 (720p).mp4: 640x352 (no detections), 10.7ms
video 1/4 (3/376) /content/drive/MyDrive/YoloV8/videos/pexels-anastasia-shuraeva-9429927 (720p).mp4: 640x352 (no detections), 10.6ms
video 1/4 (4/376) /content/drive/MyDrive/YoloV8/videos/pexels-anastasia-shuraeva-9429927 (720p).mp4: 640x352 (no detections), 10.7ms
video 1/4 (5/376) /content/drive/MyDrive/YoloV8/videos/pexels-anastasia-shuraeva-9429927 (720p).mp4: 640x352 (no detections), 10.6ms
video 1/4 (6/376) /content/drive/MyDrive/YoloV8/videos/pexels-anastasia-shuraeva-9429927 (720p).mp4: 640x352 (no detections), 10.6ms
video 1/4 (7/376) /content/drive

In [15]:
!cp -r /content/drive/MyDrive/YoloV8/runs/detect/predict3 /content/drive/MyDrive/YoloV8/output